# Concrete Compressive Strength Dataset 3D Visualization Lab

## Dataset Overview
The Concrete Compressive Strength dataset consists of 1,030 samples of concrete mixtures, each described by seven ingredient proportions and the curing age. The target variable is the measured compressive strength (MPa), reflecting material performance.

- **Source:** UCI Machine Learning Repository
- **Features:** Cement, Slag, FlyAsh, Water, Superplasticizer, CoarseAggregate, FineAggregate, Age (days)
- **Target:** Strength (MPa)

We will create five distinct 3D visualizations to explore how mix components and curing age influence compressive strength.

In [ ]:
%%bash
pip install --quiet xlrd==2.0.1

python - << 'PYCODE'
import pandas as pd
# Load the dataset
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls', engine='xlrd')
df.columns = ['Cement','Slag','FlyAsh','Water','Superplasticizer','CoarseAggregate','FineAggregate','Age','Strength']
import pickle
pickle.dump(df, open('df_concrete.pkl','wb'))
PYCODE

In [ ]:
import pandas as pd
df = pd.read_pickle('df_concrete.pkl')
df.head()

## 3D Scatter Plot: Cement vs Water vs Strength

**Purpose and Rationale:** Visualize the interaction between cement and water proportions on compressive strength to understand optimal mix ratios.

**Big Takeaway:** High cement with moderate water achieves peak strength; too much water reduces strength even at high cement levels.

- **What does this image show?** Each point is a sample plotted by Cement (x), Water (y), and Strength (z).
- **What is its highlight?** The densest cluster of high-strength points lies at Cement ~300–350 kg/m³ and Water ~150–180 kg/m³.
- **What can people learn?** Optimize water-to-cement ratio for maximum strength.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(df['Cement'], df['Water'], df['Strength'], c=df['Strength'], cmap='viridis')
ax.set_xlabel('Cement (kg/m³)'); ax.set_ylabel('Water (kg/m³)'); ax.set_zlabel('Strength (MPa)')
ax.set_title('Cement vs Water vs Strength')
fig.colorbar(scatter, label='Strength (MPa)')
plt.savefig('CementWaterStrength.png', dpi=300)
plt.show()

## 3D Surface Plot: Cement vs Age vs Strength

**Purpose and Rationale:** Model how curing age and cement content jointly influence strength gain over time.

**Big Takeaway:** Higher cement mixes gain strength faster in early days, with diminishing returns after ~28 days.

- **What does this image show?** A continuous surface mapping Strength as a function of Cement (x) and Age (y).
- **What is its highlight?** The steepest strength gains occur for high cement content during the first 28 days.
- **What can people learn?** Plan project schedules around rapid early gains in high-cement mixtures.

In [ ]:
from scipy.interpolate import griddata
x, y, z = df['Cement'], df['Age'], df['Strength']
xi = np.linspace(x.min(), x.max(), 30); yi = np.linspace(y.min(), y.max(), 30)
xi, yi = np.meshgrid(xi, yi)
zi = griddata((x, y), z, (xi, yi), method='linear')
fig = plt.figure(figsize=(10,7)); ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(xi, yi, zi, cmap='plasma', linewidth=0)
ax.set_xlabel('Cement'); ax.set_ylabel('Age (days)'); ax.set_zlabel('Strength')
ax.set_title('Cement vs Age vs Strength')
fig.colorbar(surf, label='Strength (MPa)')
plt.savefig('CementAgeStrength.png', dpi=300)
plt.show()

## 3D Wireframe Plot: Water vs Superplasticizer vs Strength

**Purpose and Rationale:** Examine whether superplasticizer can offset high water content's negative effect on strength.

**Big Takeaway:** Elevated superplasticizer allows reduced water while maintaining high strength.

- **What does this image show?** Wireframe mesh of Strength over Water and Superplasticizer.
- **What is its highlight?** Ridge where Superplasticizer >8 kg/m³ and Water <160 kg/m³ indicates optimum admixture use.
- **What can people learn?** Use enough superplasticizer to lower water without compromising strength.

In [ ]:
fig = plt.figure(figsize=(10,7)); ax = fig.add_subplot(111, projection='3d')
xi = np.linspace(df['Water'].min(), df['Water'].max(), 30)
yi = np.linspace(df['Superplasticizer'].min(), df['Superplasticizer'].max(), 30)
xi, yi = np.meshgrid(xi, yi)
zi = griddata((df['Water'], df['Superplasticizer']), df['Strength'], (xi, yi), method='linear')
ax.plot_wireframe(xi, yi, zi, color='teal')
ax.set_xlabel('Water'); ax.set_ylabel('Superplasticizer'); ax.set_zlabel('Strength')
ax.set_title('Water vs Superplasticizer vs Strength')
plt.savefig('WaterSuperplasticizerStrength.png', dpi=300)
plt.show()

## 3D Line Plot: Monthly Avg Cement vs Strength

**Purpose and Rationale:** Aggregate to monthly bins to reveal long-term trends without overplotting.

**Big Takeaway:** Strength plateaus after month 3, indicating limited gains from extended curing.

- **What does this image show?** Colored line tracing monthly-averaged Cement and Strength over time.
- **What is its highlight?** Noticeable flattening of strength gains beyond the third month.
- **What can people learn?** Optimize curing schedules to focus on early strength development.

In [ ]:
df['Month'] = (df['Age']//30)+1
monthly = df.groupby('Month').agg({'Cement':'mean','Strength':'mean'}).reset_index()
norm = plt.Normalize(monthly['Month'].min(), monthly['Month'].max())
colors = cm.inferno(norm(monthly['Month']))
fig=plt.figure(figsize=(10,7)); ax=fig.add_subplot(111, projection='3d')
for i in range(len(monthly)-1):
    xs=monthly.loc[i:i+1,'Month']; ys=monthly.loc[i:i+1,'Cement']; zs=monthly.loc[i:i+1,'Strength']
    ax.plot(xs,ys,zs,color=colors[i],linewidth=3)
mappable=cm.ScalarMappable(norm=norm,cmap='inferno'); mappable.set_array([])
fig.colorbar(mappable,label='Month')
ax.set_xlabel('Month'); ax.set_ylabel('Avg Cement'); ax.set_zlabel('Avg Strength')
ax.set_title('Monthly Cement vs Strength')
plt.savefig('MonthlyCementStrengthLine.png', dpi=300)
plt.show()

## 3D Bar Plot: Aggregate Ratios vs Strength

**Purpose and Rationale:** Compare average strength for binned ratios of Fine vs Coarse Aggregate to find optimal balance.

**Big Takeaway:** Mid-range aggregate proportions yield the highest strength, not the extremes.

- **What does this image show?** Bars showing mean Strength for bins of Fine and Coarse Aggregate.
- **What is its highlight?** Peak bar near moderate Fine and Coarse values indicates balanced mix superiority.
- **What can people learn?** Avoid extreme aggregate ratios; aim for moderate balance.

In [ ]:
fa_bins=pd.cut(df['FineAggregate'],4); ca_bins=pd.cut(df['CoarseAggregate'],4)
grp=df.groupby([fa_bins,ca_bins])['Strength'].mean().reset_index()
grp['fa_mid']=grp['FineAggregate'].apply(lambda x:x.mid)
grp['ca_mid']=grp['CoarseAggregate'].apply(lambda x:x.mid)
x=grp['fa_mid']; y=grp['ca_mid']; z=np.zeros_like(x)
dx=(x.max()-x.min())/6; dy=(y.max()-y.min())/6; dz=grp['Strength']
fig=plt.figure(figsize=(10,7)); ax=fig.add_subplot(111, projection='3d')
ax.bar3d(x,y,z,dx,dy,dz,shade=True)
ax.set_xlabel('Fine Aggregate'); ax.set_ylabel('Coarse Aggregate'); ax.set_zlabel('Strength')
ax.set_title('Aggregate Ratios vs Strength')
plt.savefig('AggregateStrengthBar.png', dpi=300)
plt.show()